In [14]:
import numpy as np
import sys,os
sys.path.append("..")
from glob import glob
from shutil import rmtree
import pandas as pd
from natsort import natsorted
import json

In [31]:
def vistype(desc: pd.Series):
    visname = desc.strip('.png')
    visname = visname.strip('.jpg')
    json_file = os.path.join('/netpool/homes/wangyo/Dataset/VisQA/merged/qa', visname + '.json')
    json_file = open(json_file)
    visjson = json.load(json_file)
    json_file.close()
    return visjson['vistype']

def excludeImage(desc: pd.Series, typen):
    src = f"/netpool/homes/wangyo/Projects/scanpath-uncertainty/VisQA+/included_{typen}"
    if not os.path.exists(src): return desc
    filelist=pd.read_csv(src, header=None)

    df = pd.DataFrame(columns=desc.columns)

    for id, row in filelist.iterrows():
        df = df.append(desc[desc['image_name'] == row[0]])

    return df


base_path = '/netpool/homes/wangyo/Dataset/VisQA/eyetracking/Results'
subjects_path = glob(os.path.join(
    f"{base_path}/*/", "*", ""))
subjects_path = [os.path.abspath(path) for path in subjects_path]

good_sum = 0
good_true = 0
bad_sum = 0
bad_true = 0
for path in subjects_path:
    pname = path.split('/')[-1]
    #good = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_0.csv"
    fgood = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_0.9degree.csv"
    fbad = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_1degree.csv"
    if(not os.path.exists(fgood) or not os.path.exists(fbad)): continue
    tmp_good=pd.read_csv(fgood)
    tmp_bad=pd.read_csv(fbad)
    tmp_good = tmp_good[tmp_good['stage'] == 'enc']
    tmp_bad = tmp_bad[tmp_bad['stage'] == 'enc']

    # exclude the bad images out
    #excludeImage(tmp_good, 'bar')
    excludeImage(tmp_good, 'line')
    #excludeImage(tmp_good, 'scatter')
    #excludeImage(tmp_good, 'pie')

    # apply the vistype
    tmp_good['image_name'] = tmp_good['image_name'].apply(vistype)
    tmp_good = tmp_good[tmp_good['image_name'] == 'line']
    tmp_bad['image_name'] = tmp_bad['image_name'].apply(vistype)
    tmp_bad = tmp_bad[tmp_bad['image_name'] == 'line']

    gtrue = (tmp_good['contains']==True).sum()
    gfalse = (tmp_good['contains']==False).sum()
    btrue = (tmp_bad['contains']==True).sum()
    bfalse = (tmp_bad['contains']==False).sum()
    good_sum += gtrue
    good_sum += gfalse
    good_true += gtrue
    bad_sum += btrue
    bad_sum += bfalse
    bad_true += btrue


print(good_sum, bad_sum)
print(good_true/good_sum)
print(bad_true/bad_sum)

32685 34093
0.875202692366529
0.8932625465638108


In [23]:
np.tan(0.1)

0.10033467208545055

# AVG
## 0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5
### 55, 61, 66, 70, 74, 77, 81, 84.7, 88
## 0.6, 0.7, 0.8, 0.9, 1.0
### 90.5, 92.5, 94, 95.1, 96

# Scatter
## 0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5
### 46, 52, 58, 63, 68, 72, 77, 82, 86
## 0.6, 0.7, 0.8, 0.9, 1.0
### 89.5, 92, 94.1, 95.4, 96.4

# line
## 0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5
### 41, 46, 51, 55, 59, 63, 67, 71, 76
## 0.6, 0.7, 0.8, 0.9, 1.0
### 79.5, 82.8, 85.3, 87.5, 89.3

# bar
## 0,   0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5
### 55.7, 62.3, 68.2, 72.9, 77.1, 80.4, 83.5, 87.9, 91.1

## 0.6, 0.7, 0.8, 0.9, 1.0
### 93.5, 95.1, 96.1, 97.0, 97.8

# pie
## 0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5
### 64, 67, 71, 74, 77, 79, 83, 85, 88
## 0.6, 0.7, 0.8, 0.9, 1.0
### 89.6, 91.4, 93.0, 94.0, 94.7